In [290]:
import os
import shutil
import numpy as np
from sklearn.svm import SVC
from skimage import feature,data
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator,load_img,array_to_img,img_to_array
import pandas as pd
from skimage.transform import resize
import warnings
warnings.filterwarnings("ignore")

In [196]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest',
    horizontal_flip=True)

In [550]:
Class = ["cat", "dog"]

def generate_image(nums):
    for animal in Class:
        img = load_img(f'images/{animal}.jpg')
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)
        dir_name = f"temp_{animal}"
        if not os.path.isdir(dir_name):
            os.mkdir(dir_name)
        else:
            shutil.rmtree(dir_name)
            os.mkdir(dir_name)
        for _ in range(nums):
            next(datagen.flow(x, batch_size=1, save_to_dir=dir_name,
                              save_prefix=animal, save_format='jpeg'))
            
            
def get_feature(file):
    img = data.imread(file,as_grey=True)
    if img.shape != (100, 100):
        img = resize(img, (100,100), anti_aliasing=True)
    edges = feature.canny(img, sigma=3).reshape(-1)
    return edges
    
def generate_feature(nums):
    generate_image(nums)
    X,Y=[],[]
    for animal in Class:
        dir_name = f"temp_{animal}"
        for images in os.listdir(dir_name):
            edges= get_feature(os.path.join(dir_name, images))
            X.append(edges)
            Y.append(animal)
    return X,Y


def remove_temp_data():
    for animal in Class:
        dir_name = f"temp_{animal}"
        shutil.rmtree(dir_name)
        

# the image will created dynamiclly
def train_model(num_of_images):
    x,y = generate_feature(num_of_images)
    model = SVC(kernel="rbf",C=1e6)
    model.fit(x,y)
    remove_temp_data()
    return model

In [555]:
model = train_model(100)

In [556]:
test_cat = get_feature("images/test_cat.png")
test_dog = get_feature("images/test_dog.jpg")

In [557]:
model.predict([test_cat])

array(['cat'], dtype='<U3')

In [558]:
model.predict([test_dog])

array(['dog'], dtype='<U3')